# 三大法人期貨淨未平倉量口數  
* 抓取目標包含大台、小台、微型台指期貨，以及金融期貨、電子期貨  
* 三大法人（外資、投信、自營商）期貨未平倉口數淨額  
* 使用 finlab api 取得資料  

In [26]:
import os
import sys
from pathlib import Path
import pandas as pd
import duckdb

In [27]:
from finlab import data

In [28]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

## 公用參數設定

In [29]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

## 外部資料讀取  

In [31]:
# 外資期貨未平倉口數淨額
net_oi = data.get('futures_institutional_investors_trading_summary:多空未平倉口數淨額', save_to_storage=True)

In [ ]:
# 選取特定欄位
target_cols = [
    "臺股期貨_外資及陸資", "臺股期貨_投信", "臺股期貨_自營商",  
    "小型臺指期貨_外資及陸資", "小型臺指期貨_投信", "小型臺指期貨_自營商",
    "微型臺指期貨_外資及陸資", "微型臺指期貨_投信", "微型臺指期貨_自營商",
    "金融期貨_外資及陸資", "金融期貨_投信", "金融期貨_自營商",
    "電子期貨_外資及陸資", "電子期貨_投信", "電子期貨_自營商"
]

net_oi_df = net_oi.loc[:, target_cols]
net_oi_df.reset_index(inplace=True)

# column name 含有 "外資及陸資" 的欄位，將其名稱改為 "外資"
net_oi_df.columns = net_oi_df.columns.str.replace("外資及陸資", "外資")

# 將 date 欄位 rename 為"Date" 欄位轉換為 datetime 格式
net_oi_df.rename(columns={"date": "Date"}, inplace=True)
net_oi_df["Date"] = pd.to_datetime(net_oi_df["Date"])

# 取近2個月資料
net_oi_df = net_oi_df[net_oi_df["Date"] >= pd.Timestamp.now() - pd.DateOffset(months=2)]

print(net_oi_df.shape)
net_oi_df.tail(3)


## 資料留存ＤＢ

In [33]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [34]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [35]:
table_name = "tw_futures_institutional_investors_trading_summary"

In [ ]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    net_oi_df, 
    table_name, 
    date_column='Date',
    if_exists='replace'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

In [ ]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

In [38]:
# 關閉資料庫連線
conn_duckdb.close()